<html><head><meta content="text/html; charset=UTF-8" http-equiv="content-type"><style type="text/css">ol</style></head><body class="c5"><p class="c0 c4"><span class="c3"></span></p><p class="c2 title" id="h.rrbabt268i6e"><h1>CaImAn&rsquo;s Demo pipeline, </h1></p><p class="c0"><span class="c3">This notebook will help to demonstrate the process of CaImAn and how it uses different functions to denoise, deconvolve and demix neurons from a Calcium Imaging Video. </span></p>
<p><img src="docs/img/quickintro.png" /></p>
<p class="c0"><span class="c3">More information can be found in CaImAn&rsquo;s documentation. </span></p>
</html>



In [78]:
#!/usr/bin/env python
from __future__ import division
from __future__ import print_function
from builtins import zip
from builtins import str
from builtins import map
from builtins import range
from past.utils import old_div
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')    
except:
    print('Not IPYTHON')
import numpy as np
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib qt')   
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.components_evaluation import estimate_components_quality_auto
from caiman.base.rois import register_ROIs
from caiman.base.traces import trace
import os
import cv2
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import matplotlib.pyplot as plt


try:
    cv2.setNumThreads(1)
except:
    print('Open CV is naturally single threaded')
import bokeh.plotting as bpl
bpl.output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [15]:
import caiman as cm
import numpy as np
from caiman.source_extraction import cnmf
from caiman.utils.visualization import inspect_correlation_pnr
from caiman.base.rois import register_ROIs
import pickle

In [16]:
root = '/home/sebastian/Desktop/cnmfs/32364'
roots = []
paths = {}
frames_paths = {}

for i in os.listdir(root):
    if int(i[-2:])<15:
        root_name = (os.path.join(root,i))
        roots.append(root_name)
        for f in os.listdir(os.path.join(root,i)):
            if 'cnmf.pkl' in f: 
                paths[root_name]=(os.path.join(root,i,f))
            if 'frames.npy' in f:
                frames_paths[root_name]=(os.path.join(root,i,f))

roots.sort()
print(roots)

['/home/sebastian/Desktop/cnmfs/32364/20170710', '/home/sebastian/Desktop/cnmfs/32364/20170711', '/home/sebastian/Desktop/cnmfs/32364/20170712', '/home/sebastian/Desktop/cnmfs/32364/20170713', '/home/sebastian/Desktop/cnmfs/32364/20170714']


### (Re)start cluster.  

In [17]:
frate = 10 # movie frame rate
gSig = 3   # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = 10  # average diameter of a neuron

min_corr = .3 # min correlation of peak (from correlation image)
min_pnr = 5 # min peak to noise ratio
min_SNR = 3 # adaptive way to set threshold on the transient size


In [18]:
try:
    dview.terminate() # stop it if it was running
except:
    pass

c, dview, n_processes = cm.cluster.setup_cluster(backend='local', # use this one
                                                 n_processes=10,  # number of process to use, if you go out of memory try to reduce this one
                                                 )

In [19]:
root1 = roots[0]

cnm1 = cnmf.CNMF(n_processes=n_processes) 
cnm1.load(paths[root1])
frames1 = np.load(frames_paths[root1])

In [39]:
x_rest = np.zeros((cnm1.C.shape[1]))

f = 0
mode = 1
for frame in frames1:
    x_rest[f:]=mode
    f+=frame
    mode=1-mode
 
x_rest = x_rest*2-1
plt.plot(x_rest)

In [40]:
spikes = cnm1.S[:,:]
trains = np.zeros((spikes.shape[0]))

for i in range(trains.shape[0]):
    trains[i] = (spikes[i,:]*x_rest).mean()

trains = trains-trains.mean()

rest_active = np.where(trains>(trains.mean()+(1.2*np.std(trains))))[0]
test_active = np.where(trains<(trains.mean()-(1.2*np.std(trains))))[0]

plt.figure(1,figsize=(15, 15))

trc = trace(spikes[::8,:].T)
plt.subplot2grid((3,2),(0,0),colspan=2,rowspan=2)
plt.title('Spikes:')
trc.plot()

plt.subplot2grid((3,2),(2,0))
plt.title('active during rest:')
trc = trace(spikes[rest_active,:].T)
trc.plot()


plt.subplot2grid((3,2),(2,1))
plt.title('active during test:')
trc = trace(spikes[test_active,:].T)
trc.plot()

# np.save('spikes.npy',spikes)

/home/sebastian/Documents/CaImAn/caiman/base/traces.py:127: MatplotlibDeprecationWarning: The set_color_cycle function was deprecated in version 1.5. Use `.set_prop_cycle` instead.
  ax.set_color_cycle(colors)


In [64]:
cnmfs = {}
n_s = [0]

for r in roots:
    cnmfs[r] = cnmf.CNMF(n_processes=n_processes) 
    cnmfs[r].load(paths[r])
    n_s.append(cnmfs[r].C.shape[0])


### Combine cells over days:


In [76]:
matches = {}

for i,r1 in enumerate(roots[:-1]):
    matches[r1] = {}
    for j,r2 in enumerate(roots[i+1:]):
        cnm1 = cnmfs[r1]
        cnm2 = cnmfs[r2]
        matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance = register_ROIs(cnm1.A,cnm2.A,cnm1.dims,cnm1.cn_filter,cnm2.cn_filter)
        
        matches[r1][r2]=[matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance].copy()



New Pair **
FOV: 0, shape: 225,95 total cost: 54.465926
0.03588581085205078
{'recall': 0.32, 'precision': 0.7578947368421053, 'accuracy': 0.2903225806451613, 'f1_score': 0.45}
New Pair **
FOV: 0, shape: 225,195 total cost: 138.711686
0.29004836082458496
{'recall': 0.4622222222222222, 'precision': 0.5333333333333333, 'accuracy': 0.3291139240506329, 'f1_score': 0.49523809523809526}
New Pair **
FOV: 0, shape: 225,128 total cost: 77.443863
0.11381316184997559
{'recall': 0.38222222222222224, 'precision': 0.671875, 'accuracy': 0.32209737827715357, 'f1_score': 0.48725212464589235}
New Pair **
FOV: 0, shape: 225,106 total cost: 67.866583
0.06405496597290039
{'recall': 0.29333333333333333, 'precision': 0.6226415094339622, 'accuracy': 0.2490566037735849, 'f1_score': 0.3987915407854985}
New Pair **
FOV: 0, shape: 95,195 total cost: 66.344428
0.049494266510009766
{'recall': 0.5684210526315789, 'precision': 0.27692307692307694, 'accuracy': 0.2288135593220339, 'f1_score': 0.3724137931034483}
New Pai

In [77]:
match_mat = np.zeros([sum(n_s),sum(n_s)])

n_s_int = np.cumsum(n_s)

for i,k1 in enumerate(matches.keys()):
    for j,k2 in enumerate(matches[k1].keys()):
        n1 = n_s_int[i]
        n2 = n_s_int[i+j]
        #print(k1,k2,i,j+i)
        ROI1 = np.array(matches[k1][k2][0])+n1
        ROI2 = np.array(matches[k1][k2][1])+n2
        #ROI2 = np.array(matches[k1][k2][0])+n2
        match_mat[ROI1,ROI2]+=1
        match_mat[ROI2,ROI1]+=1

for n1 in n_s_int[:-1]:
    match_mat[:,n1]+=1
    match_mat[n1,:]+=1
        
        
        
plt.imshow(match_mat)

In [48]:

    
plt.figure(1,figsize=(40, 40))
    
for i,r in enumerate(roots[:-1]):
    plt.subplot(4,2,2*i+1)
    cnm1 = cnmfs[r]
    cnm2 = cnmfs[roots[i+1]]
    crd3 = cm.utils.visualization.plot_contours(cnm1.A[:,matches[r][0]], cnm1.cn_filter, thr=.9, vmax=0.95)
    plt.subplot(4,2,2*i+2)
    crd4 = cm.utils.visualization.plot_contours(cnm2.A[:,matches[r][1]], cnm2.cn_filter, thr=.9, vmax=0.95)
    

    
    

/home/sebastian/Documents/CaImAn/caiman/utils/visualization.py:941: UserWarning: The way to call utilities.plot_contours has changed. Look at the definition for more details.
  warn("The way to call utilities.plot_contours has changed. Look at the definition for more details.")


In [35]:
cnmfs

{'/home/sebastian/Desktop/cnmfs/32364/20170710': <caiman.source_extraction.cnmf.cnmf.CNMF at 0x7f63c01ad390>,
 '/home/sebastian/Desktop/cnmfs/32364/20170711': <caiman.source_extraction.cnmf.cnmf.CNMF at 0x7f63c0112240>,
 '/home/sebastian/Desktop/cnmfs/32364/20170712': <caiman.source_extraction.cnmf.cnmf.CNMF at 0x7f63c0bcb8d0>,
 '/home/sebastian/Desktop/cnmfs/32364/20170713': <caiman.source_extraction.cnmf.cnmf.CNMF at 0x7f63c02c8f60>,
 '/home/sebastian/Desktop/cnmfs/32364/20170714': <caiman.source_extraction.cnmf.cnmf.CNMF at 0x7f63c0117c18>}

In [111]:
ms

[[array([  5,   6,  11,  23,  24,  28,  31,  34,  41,  63,  64,  66,  67,
          68,  69,  71,  74,  79,  80,  82,  83,  85,  87,  88,  91,  93,
          95,  98, 100, 101, 103, 104, 105, 106, 108, 109, 111, 114, 115,
         117, 118, 122, 124, 126, 132, 139, 140, 142, 143, 148, 151, 152,
         154, 158, 159, 161, 162, 167, 169, 172, 173, 178, 192, 193, 203,
         206, 208, 211, 213, 214, 216, 218, 223, 224]),
  array([  7,   4,   6,  12,  11,  10,  99,  14,   8,  29,  37,  41,  16,
          30,  19,  36,  45,  48,  40,  52,  42,  43,  60,  47,  44,  20,
          46,  50,  53,  49,  62,  61,  64,  63,  51,  54,  65,  59,  57,
          67,  55,  66,  68,  69,  21,  70,  71,  23,  24,  22,  72,  73,
          74,  75,  33,  32,  81,  38,  77,  79,  39, 100,  76,  82,  98,
          80,  97,  84,  85,  90,  94,  95,  27,  56])],
 [array([  2,   5,   6,  10,  11,  12,  16,  17,  24,  25,  26,  35,  37,
          40,  44,  48,  50,  64,  65,  66,  67,  69,  71,  72,  76,  78,

In [26]:
import matplotlib.pyplot as plt
plt.figure(2,figsize=(20, 15))

plt.subplot(1,2,1)
crd3 = cm.utils.visualization.plot_contours(cnm1.A[:,matched_ROIs1], cnm1.cn_filter, thr=.9, vmax=0.95)
plt.subplot(1,2,2)
crd4 = cm.utils.visualization.plot_contours(cnm2.A[:,matched_ROIs2], cnm2.cn_filter, thr=.9, vmax=0.95)

NameError: name 'matched_ROIs1' is not defined

In [27]:
%matplotlib qt

spikes1 = cnm1.S[matched_ROIs1[::2],:]
spikes2 = cnm2.S[matched_ROIs2[::2],:]

trc1 = trace(spikes1.T)
trc2 = trace(spikes2.T)

plt.figure(3,figsize=(20, 20))


plt.subplot(1,2,1)
plt.title('spikes day 1:')
trc1.plot(cmap = plt.cm.tab20b)


plt.subplot(1,2,2)
plt.title('spikes day 2:')
trc2.plot(cmap = plt.cm.tab20b)


NameError: name 'matched_ROIs1' is not defined

In [ ]:
cnm1.S[0].shape

### Stop cluster

In [ ]:
#%%
cm.stop_server(dview=dview)

In [ ]:
##TODO:

# from caiman import minisom

# # Initialization and training
# som = minisom.MiniSom(420, 600, cnm1.S.shape[1], sigma=10.0, learning_rate=0.5)
# som.random_weights_init(cnm1.S)
# print("Training...")
# som.train_random(cnm1.S, 10)  # random training
# print("\n...ready!")